# Podstawy Sztucznej Inteligencji 2018/2019



Prosze uzupelnic kod tam gdzie znajduje napis `YOUR CODE HERE` lub 'YOUR ANSWER HERE'.

Warto zresetowac 'kernel' i sprawdzic czy caly notatnik uruchamiany od poczatku nie daje bledow.

---

## Naiwny Bayes

Notatnik ten zawiera implementacją naiwnego klasyfikatowa Bayesa w numpy z porównaniem wyników z klasą ``GaussianNB`` z biblioteki `sklearn`.

Z twierdzenia Bayesa mamy:

$$P(J=j|X=x)=\frac{P(X=x|J=j)P(J=j)}{\sum_{j\in\{1,\ldots L\}}P(X=x|J=j)P(J=j)}$$

$$ p_j(x)=\frac{p_jf_j(x)}{f(x)} $$

Wykorzystamy te formuły w budowie klasyfikatora [Bayesa](http://books.icse.us.edu.pl/runestone/static/ai/KlasyfikacjaWOparciuPodobienstwoDoWzorca/OptymalnyKlasyfikatorStatystyczny.html).

In [418]:
import numpy as np 
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split
from sklearn import datasets 
# importujemy zbiór danych
iris = datasets.load_iris()

gnb = GaussianNB()

# Zbiór obiektów
X = iris.data

# Wektor poprawnej klasyfikacji obiektów
y = iris.target
y = np.array(y)

# Dzielimy losowo zbiór na dwie części
train, test, train_targets, test_targets = train_test_split(X, y,
                                 test_size=0.31)
# Uczymy klasyfikator
clf = gnb.fit(train, train_targets)

#Testujemy
Z = clf.predict(test)

In [419]:
train.shape,test.shape

((103, 4), (47, 4))

### Wyznacz unikalne klasy na zbiorze trenującym.

In [420]:
Nlabels = np.unique(train_targets)

In [421]:
np.testing.assert_array_equal(Nlabels,np.array([0, 1, 2]))

### Jaki jest procent poprawych odpowiedzi?

Implementacja ``GaussianNB`` daje tą odpowiedz w następujący sposób:

In [422]:
clf.score(test,test_targets)

0.9148936170212766

In [423]:
correct = np.sum(Z == test_targets)/ len(Z)
print(correct)

0.9148936170212766


In [424]:
assert correct==clf.score(test,test_targets)

### Które odpowiedzi są złe:

In [425]:
bad_idx = np.where(Z!=test_targets)
bad_idx

(array([ 2,  3, 12, 33]),)

### Parametry klasyfikatora

In [426]:
clf.theta_

array([[4.9875    , 3.375     , 1.44      , 0.235     ],
       [5.990625  , 2.821875  , 4.29375   , 1.309375  ],
       [6.52580645, 3.00322581, 5.43548387, 1.99032258]])

In [427]:
clf.sigma_

array([[0.11109375, 0.120375  , 0.0289    , 0.010775  ],
       [0.25272461, 0.09108399, 0.16683594, 0.03459961],
       [0.33094693, 0.11063476, 0.20938606, 0.08216442]])

Średnie wartości dla każdego atrybutu dla klasy numer 2

In [428]:
ith = 2
np.mean(train[train_targets == ith],axis=0)

array([6.52580645, 3.00322581, 5.43548387, 1.99032258])

Wariancje dla każdego atrybytu dla klasy nr 2

In [429]:
np.var(train[train_targets == ith],axis=0)

array([0.33094693, 0.11063476, 0.20938606, 0.08216441])

### Implementacja naiwnego klasyfikatora Bayesa

**1\. Oblicz częstość występowania poszczególnych klasy $j$ w zbiorze testowym $p_{j}$**

Jest to prawdopodobieństwo *a priori*.

In [430]:
p = [np.sum(train_targets == 0)/ len(train_targets),
     np.sum(train_targets == 1)/ len(train_targets),
     np.sum(train_targets == 2)/ len(train_targets)]
print(p)

[0.3883495145631068, 0.3106796116504854, 0.30097087378640774]


In [431]:
np.testing.assert_allclose(p,clf.class_prior_)
### BEGIN HIDDEN TEST
np.testing.assert_allclose( gnb.class_count_/np.sum(gnb.class_count_), gnb.class_prior_ )
### END HIDDEN TEST

**2\. Oblicz wartość średnią dla każdej cechy z każdej klasy.**

Niech $\mu_{ij}$ oznacza  wartość średnią dla $j$-tej zmiennej w $i$-tej klasie, wtedy:
   
$$ \mu_{ij} =  \langle x_j \rangle_{ \forall x_j \in \;\mathrm{label} \;
i } $$

In [432]:
mu = []
for i in range(3):
    klasa = train[train_targets==i]
    mu.append(np.mean(klasa, axis = 0))
mu = np.array(mu)
print(mu)

[[4.9875     3.375      1.44       0.235     ]
 [5.990625   2.821875   4.29375    1.309375  ]
 [6.52580645 3.00322581 5.43548387 1.99032258]]


In [433]:
np.testing.assert_allclose(mu,clf.theta_)

In [434]:
mu.shape

(3, 4)

**3\. Oblicz wariancję $j$-tej zmiennej w $i$-tej klasie.**

   $$ \sigma_{ij}^2 = \mathrm{Var} [x_j] _ {\;\;{ \forall x_j \in \mathrm{class}\; i}} $$

In [435]:
sigma2 = []
for i in range(3):
    klasa = train[train_targets==i]
    sigma2.append(np.var(klasa, axis = 0))
sigma2 = np.array(sigma2)
print(sigma2)

[[0.11109375 0.120375   0.0289     0.010775  ]
 [0.25272461 0.09108398 0.16683594 0.03459961]
 [0.33094693 0.11063476 0.20938606 0.08216441]]


In [436]:
np.testing.assert_almost_equal(sigma2,clf.sigma_)

**4\. Oblicz prawdopodobieństwo *a posteriori* klasy $i$ dla danego wektora zmiennych  $\mathbf{x}$.**

Niech $k$ oznacza liczbę cech (zmiennych). W naszym przypadku mamy $k=4$.

Dla $i$-tej klasy mamy:


$$ P_i(\mathbf{x}) \simeq p_i f_i = p_i \frac{1}{\sqrt{(2\pi)^k\Pi_{j=1}^k\sigma_{ij}^2} } e^{ -\displaystyle\sum_{j=1}^{k}\frac{(x_j-\mu_{ij})^2}{2\sigma_{ij}^2} }
 $$
 
By otrzymać prawdopodobieństwa należy unormować $P_i$ dla każdego przypadku tak by suma $\sum_i P_i(\mathbf{x})=1$

In [ ]:
x = test
k = test.shape[1]
P = []
for l in range(len(x)):
    P.append([])
    for i in range(k-1):
        low = np.sqrt((2*np.pi)**k * np.prod(sigma2[i]))
        high = np.sum((((x[l] - mu[i])**2) / (2* sigma2[i])))
        P[l].append(p[i] * (1 / low) * np.e ** (-high))
    P[l] /= np.sum(P[l])
        
P = np.array(P)


In [449]:
sigma2.shape,x.shape,P.shape,np.unique(test_targets)

((3, 4), (47, 4), (47, 3), array([0, 1, 2]))

In [450]:
np.testing.assert_almost_equal(P,clf.predict_proba(x))

**5\. Wyznacz klasę dla której prawdopodobieństwo *a posteriori* jest największe** 

 $i$ : $\quad\textrm{ gdy } P_i(x)=\max_{1\leq j\leq L} P_j(x)$


In [451]:
prediction = np.argmax(P, axis = 1)

In [452]:
np.testing.assert_equal( prediction, clf.predict(test))

In [453]:
print(clf.predict(test) )
print(prediction)

[1 0 2 2 1 2 1 0 0 1 2 0 1 2 1 2 0 2 2 1 0 2 1 0 2 1 0 2 2 1 2 0 2 2 1 1 2
 1 2 1 1 2 1 2 0 2 2]
[1 0 2 2 1 2 1 0 0 1 2 0 1 2 1 2 0 2 2 1 0 2 1 0 2 1 0 2 2 1 2 0 2 2 1 1 2
 1 2 1 1 2 1 2 0 2 2]


In [454]:
clf.score(test,test_targets)

0.9148936170212766